## Compact and full LLM szerver for the Chrome Extension

#### The elte-ik-llm-dlite notebook contains the installation and the experiments for the project.
#### This notebook is the practical runnable server.

### (1) Load the pretrained model

Load and prepare the dlite-v2-1_5b LLM to the GPU

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="cuda", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

### (2) The REST server

Initialise Flask REST CORS server for recive the process requests

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import subprocess
import re

app = Flask(__name__)
CORS(app)

def req_length(text):
    tokens = tokenizer(text)
    tokens_length = len(tokens.attention_mask)
    print('LOG: Input length: ' + str(tokens_length))
    return tokens_length

def req_to_json(title, text):
    return jsonify({title : text})

def req_process(command, text):
    req = command + ' ' + text
    print('LOG: ' + req)
    return generate_text(req)

def req_clean(text):
    cleaned_text = text.replace('\n', ' ').replace('\r', ' ')
    cleaned_text = re.sub(r'[^\w\s.]', '', cleaned_text)
    return cleaned_text

@app.route('/sum', methods=['POST'])
def sum_req():
    data = request.get_json()
    print(data)
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Summarize in one sentence:', text))
        #return req_to_json('response', "TEST TEXT")
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/key', methods=['POST'])
def key_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give some keywords from this text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/title', methods=['POST'])
def title_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give the most important word in the text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/length', methods=['POST'])
def length_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    ret = {'response': req_length(text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/May/2024 20:41:47] "POST /length HTTP/1.1" 200 -


LOG: Input length: 242


127.0.0.1 - - [11/May/2024 20:43:03] "POST /length HTTP/1.1" 200 -


LOG: Input length: 242


...

127.0.0.1 - - [12/May/2024 01:36:55] "POST /sum HTTP/1.1" 200 -


{'text': 'The MiG-21, a legendary Soviet-designed supersonic fighter aircraft, holds a revered place in aviation history. Introduced in the 1950s, it became one of the most widely produced jet fighters in the world, serving with over 60 countries across several decades. Its sleek delta-wing design and powerful engines gave it impressive speed and agility, earning it the nickname "Fishbed" in NATO nomenclature.  Renowned for its simplicity and reliability, the MiG-21 was designed to be easy to maintain and operate, making it a favorite among air forces with limited resources. Its delta wing configuration allowed for quick take-offs and landings, even from short runways, making it well-suited for operations in a variety of environments.  The MiG-21 saw extensive combat action during the Cold War and in numerous conflicts around the world. Its combat record includes engagements in Vietnam, the Middle East, and Africa, where its speed and maneuverability often gave it an edge over its adve

127.0.0.1 - - [12/May/2024 01:57:57] "POST /sum HTTP/1.1" 200 -


{'text': "Hello! Here's a 200-word text about cats:  Cats, beloved companions to millions around the world, captivate us with their grace, independence, and mysterious allure. Domesticated for thousands of years, these enigmatic creatures have woven themselves into the fabric of human society, earning their place as cherished pets and valued members of the family.  Known for their agility and keen hunting instincts, cats are skilled predators capable of navigating even the most precarious of environments with ease. Their lithe bodies, sharp claws, and acute senses make them formidable hunters, capable of stalking prey with silent precision.  Yet, despite their predatory nature, cats also possess a softer side. They are affectionate animals, forming strong bonds with their human companions and often seeking out cuddles and attention. Their soothing purrs and gentle nuzzles bring comfort and joy to those fortunate enough to share their homes.  Cats come in a variety of breeds, colors, an

127.0.0.1 - - [12/May/2024 02:02:01] "POST /sum HTTP/1.1" 200 -
